# **Homework 2-1 Phoneme Classification**

## 美国国防部高级研究计划局TIMIT语音连续语料库(TIMIT)
TIMIT阅读语音语料库旨在为语音语音知识的获取和自动语音识别系统的开发与评价提供语音数据。

这个作业是一个多级分类任务，
我们将训练一个深度神经网络分类器，从语音语料库TIMIT中预测每一帧的音素。

详情请参考:https://academictorrents.com/details/34e2b78745138186976cbc27939b1b34d18bd5b3

## 导入相关包

In [1]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

## 辅助函数

In [2]:
#check device
def get_device():
    return 'cuda' if torch.cuda.is_available() else 'cpu'

# fix random seed
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  
    np.random.seed(seed)  
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

## 准备数据
从`.npy`文件(Numpy array)中读取训练数据和测试数据

In [3]:
print('Loading data ...')

data_root='./timit_11/'
train = np.load(data_root + 'train_11.npy')
train_label = np.load(data_root + 'train_label_11.npy')
test = np.load(data_root + 'test_11.npy')

print('Size of training data: {}'.format(train.shape))
print('Size of testing data: {}'.format(test.shape))

Loading data ...
Size of training data: (1229932, 429)
Size of testing data: (451552, 429)


## 设置超参数
`config`中包含了训练所需要的参数以及模块存储的地址

In [4]:
# fix random seed for reproducibility
same_seeds(0)

# get device 
device = get_device()
print(f'DEVICE: {device}')

config = {
    'val_ratio': 0.2, #validation_data的占比，可调的超参数
    'batch_size': 64,
    'num_epoch': 20,  # number of training epoch
    'learning_rate': 0.0001, # learning rate
    'model_path': './model.ckpt'   
}

DEVICE: cpu


## 创建DataSet
这是搭建一个网络所必须的准备工作，数据就是支撑机器成长的必备燃料。

In [5]:
#规范的做法是封装一个类，进行数据的相关处理
class TIMITDataset(Dataset):
    def __init__(self, X, y=None):             
        self.data = torch.from_numpy(X).float()#Creates a Tensor from a numpy.ndarray.
        if y is not None:
            y = y.astype(np.int)
            self.label = torch.LongTensor(y)
        else:
            self.label = None

    def __getitem__(self, idx):
        if self.label is not None:
            return self.data[idx], self.label[idx]
        else:
            return self.data[idx]

    def __len__(self):
        return len(self.data)

接下来这一步用于在train_data中划分出validation_data

In [6]:
percent = int(train.shape[0] * (1 - config['val_ratio']))
train_x, train_y, val_x, val_y = train[:percent], train_label[:percent], train[percent:], train_label[percent:]
print('Size of training set: {}'.format(train_x.shape))
print('Size of validation set: {}'.format(val_x.shape))

Size of training set: (983945, 429)
Size of validation set: (245987, 429)


## 创建DataLoader

In [7]:
train_set = TIMITDataset(train_x, train_y)
val_set = TIMITDataset(val_x, val_y)
train_loader = DataLoader(train_set, batch_size=config['batch_size'], shuffle=True) #only shuffle the training data
val_loader = DataLoader(val_set, batch_size=config['batch_size'], shuffle=False)

C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_11348/3831140761.py:6: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = y.astype(np.int)


## 创建分类模型

In [8]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.layer = nn.Sequential(
            nn.Linear(429, 1024),
            nn.Sigmoid(),
            nn.Linear(1024, 512),
            nn.Sigmoid(),
            nn.Linear(512, 128),
            nn.Sigmoid(),
            nn.Linear(128, 39)
            
        )

    def forward(self, x):
        
        return self.layer(x)

## 训练模块

In [9]:
# create model, define a loss function, and optimizer
model = Classifier().to(device)
criterion = nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate'])

## 开始训练

In [11]:
best_acc = 0.0
for epoch in range(config['num_epoch']):
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0

    # training
    model.train() # set the model to training mode
    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad() 
        outputs = model(inputs) 
        batch_loss = criterion(outputs, labels)
        _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
                                              #按照习惯，有时候单个独立下划线是用作一个名字，来表示某个变量是临时的或无关紧要的。
        batch_loss.backward() 
        optimizer.step() 

        train_acc += (train_pred.cpu() == labels.cpu()).sum().item()
        train_loss += batch_loss.item()

    # validation
    if len(val_set) > 0:
        model.eval() # set the model to evaluation mode
        with torch.no_grad():
            for i, data in enumerate(val_loader):
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                batch_loss = criterion(outputs, labels) 
                _, val_pred = torch.max(outputs, 1) 
            
                val_acc += (val_pred.cpu() == labels.cpu()).sum().item() # get the index of the class with the highest probability
                val_loss += batch_loss.item()

            print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f} | Val Acc: {:3.6f} loss: {:3.6f}'.format(
                epoch + 1, config['num_epoch'], train_acc/len(train_set), train_loss/len(train_loader), val_acc/len(val_set), val_loss/len(val_loader)
            ))

            # if the model improves, save a checkpoint at this epoch
            if val_acc > best_acc:
                best_acc = val_acc
                torch.save(model.state_dict(), config['model_path'])
                print('saving model with acc {:.3f}'.format(best_acc/len(val_set)))
    else:
        print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.format(
            epoch + 1, config['num_epoch'], train_acc/len(train_set), train_loss/len(train_loader)
        ))

# if not validating, save the last epoch
if len(val_set) == 0:
    torch.save(model.state_dict(), config['model_path'])
    print('saving model at last epoch')

[001/020] Train Acc: 0.594383 Loss: 1.330666 | Val Acc: 0.628639 loss: 1.211098
saving model with acc 0.629
[002/020] Train Acc: 0.644506 Loss: 1.154064 | Val Acc: 0.660421 loss: 1.101216
saving model with acc 0.660
[003/020] Train Acc: 0.672216 Loss: 1.052246 | Val Acc: 0.676300 loss: 1.038718
saving model with acc 0.676
[004/020] Train Acc: 0.691347 Loss: 0.983104 | Val Acc: 0.685154 loss: 1.001852
saving model with acc 0.685
[005/020] Train Acc: 0.705615 Loss: 0.931955 | Val Acc: 0.689301 loss: 0.984177
saving model with acc 0.689
[006/020] Train Acc: 0.716344 Loss: 0.891687 | Val Acc: 0.694516 loss: 0.964627
saving model with acc 0.695
[007/020] Train Acc: 0.725881 Loss: 0.857907 | Val Acc: 0.697720 loss: 0.951889
saving model with acc 0.698
[008/020] Train Acc: 0.733717 Loss: 0.829495 | Val Acc: 0.696687 loss: 0.949866
[009/020] Train Acc: 0.741151 Loss: 0.803701 | Val Acc: 0.699374 loss: 0.944832
saving model with acc 0.699
[010/020] Train Acc: 0.748050 Loss: 0.781106 | Val Acc: 

## 测试
创建一个测试集，用来测试我们刚刚保存的模型

In [12]:
# create testing dataset
test_set = TIMITDataset(test, None)
test_loader = DataLoader(test_set, batch_size=config['batch_size'], shuffle=False)

# create model and load weights from checkpoint
model = Classifier().to(device)
model.load_state_dict(torch.load(config['model_path']))

<All keys matched successfully>

In [13]:
predict = []
model.eval() # set the model to evaluation mode
with torch.no_grad():
    for i, data in enumerate(test_loader):
        inputs = data
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, test_pred = torch.max(outputs, 1) # get the index of the class with the highest probability

        for y in test_pred.cpu().numpy():
            predict.append(y)

将预测结果写入一个`csv`文件

In [14]:
with open('prediction.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(predict):
        f.write('{},{}\n'.format(i, y))